# Bruteforcing weak authentication for AWS Console

In [1]:
!ls

 cloudtrail00.json.gz   cloudtrail07.json.gz   cloudtrail14.json.gz
 cloudtrail01.json.gz   cloudtrail08.json.gz   cloudtrail15.json.gz
 cloudtrail02.json.gz   cloudtrail09.json.gz   cloudtrail16.json.gz
 cloudtrail03.json.gz   cloudtrail10.json.gz   cloudtrail17.json.gz
 cloudtrail04.json.gz   cloudtrail11.json.gz   cloudtrail18.json.gz
 cloudtrail05.json.gz   cloudtrail12.json.gz   cloudtrail19.json.gz
 cloudtrail06.json.gz   cloudtrail13.json.gz  'Cloudtrail Hunting.ipynb'


In [4]:
gunzip -c *.json.gz | jq -cr '.Records[]|select(.eventSource=="signin.amazonaws.com" and .eventName=="ConsoleLogin" and .responseElements.ConsoleLogin=="Failure")|[.eventTime,.sourceIPAddress,.errorMessage,.awsRegion,.userIdentity.userName, .additionalEventData.MFAUsed]|@csv'

SyntaxError: invalid syntax (<ipython-input-4-ae9e215f71a9>, line 1)

In [5]:
!gunzip -c *.json.gz | jq -cr '.Records[]|select(.eventSource=="signin.amazonaws.com" and .eventName=="ConsoleLogin" and .responseElements.ConsoleLogin=="Failure")|[.eventTime,.sourceIPAddress,.errorMessage,.awsRegion,.userIdentity.userName, .additionalEventData.MFAUsed]|@csv'

"2017-05-17T23:23:34Z","8.120.255.102","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2017-05-17T23:23:41Z","8.120.255.102","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2018-02-26T01:12:46Z","231.17.3.165","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2018-02-26T18:45:36Z","12.80.110.252","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2018-02-26T18:45:51Z","12.80.110.252","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2018-02-26T18:46:19Z","12.80.110.252","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2018-02-26T18:47:44Z","12.80.110.252","No username found in supplied account","us-east-1","HIDDEN_DUE_TO_SECURITY_REASONS","No"
"2018-02-26T19:04:39Z","12.80.110.252","No username found in supplied account","us-east-1","HIDDEN

In [6]:
!gunzip -c *.json.gz | jq -cr '.Records[]|select(.eventSource=="signin.amazonaws.com" and .eventName=="ConsoleLogin" and .responseElements.ConsoleLogin=="Success" and .sourceIPAddress=="8.120.255.102")|[.eventTime,.sourceIPAddress,.errorMessage,.awsRegion,.userIdentity.userName, .additionalEventData.MFAUsed]|@csv'

# Root access through console

In [7]:
!gunzip -c *.json.gz | jq -cr '.Records[]|select(.eventName == "ConsoleLogin" and .userIdentity.type == "Root") | [.eventTime, .eventSource, .additionalEventData.MFAUsed, .sourceIPAddress, .userAgent] |@csv' | sort | uniq

"2017-02-18T19:20:48Z","signin.amazonaws.com","Yes","255.253.125.115","Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
"2017-02-19T15:37:15Z","signin.amazonaws.com","Yes","255.253.125.115","Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
"2017-02-21T04:35:07Z","signin.amazonaws.com","Yes","255.253.125.115","Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
"2017-02-24T01:38:25Z","signin.amazonaws.com","Yes","255.253.125.115","Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
"2017-02-26T17:48:22Z","signin.amazonaws.com","Yes","255.253.125.115","Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
"2017-02-27T13:07:23Z","signin.amazonaws.com","Yes","255.253.125.115","Mozilla/5.0 (Windows NT 

# An adversary may disrupt trails in an attempt to evade defenses.

In [8]:
!gunzip -c *.json.gz | jq -cr '.Records[]|select(.eventName == "StopLogging" or .eventName == "DeleteTrail" or .eventName == "UpdateTrail")|[.eventTime, .errorMessage, .userIdentity.arn, .sourceIPAddress, .eventName, .userAgent, .awsRegion]|@csv'

"2017-08-03T19:50:59Z","User: arn:aws:iam::811596193553:user/Level6 is not authorized to perform: cloudtrail:UpdateTrail on resource: arn:aws:cloudtrail:us-west-2:811596193553:trail/cloudtrail","arn:aws:iam::811596193553:user/Level6","9.95.19.14","UpdateTrail","aws-cli/1.11.118 Python/2.7.10 Darwin/16.6.0 botocore/1.5.81","us-west-2"
"2019-06-07T11:04:25Z","User: arn:aws:iam::811596193553:user/Level6 is not authorized to perform: cloudtrail:StopLogging on resource: arn:aws:cloudtrail:us-east-1:745598359964:trail/summitroute-logs","arn:aws:iam::811596193553:user/Level6","2.251.2.5","StopLogging","aws-cli/1.16.168 Python/3.7.3rc1 Linux/4.19.0-kali3-amd64 botocore/1.12.158","us-east-1"
"2019-06-07T11:04:25Z","User: arn:aws:iam::811596193553:user/Level6 is not authorized to perform: cloudtrail:StopLogging on resource: arn:aws:cloudtrail:us-east-1:745598359964:trail/summitroute-logs","arn:aws:iam::811596193553:user/Level6","2.251.2.5","StopLogging","aws-cli/1.16.168 Python/3.7.3rc1 Linux/4.

# An adversary may attempt to retrieve secrets from secrets manager- steal certificates, sensitive data

In [9]:
!gunzip -c *.json.gz | jq -cr '.Records[]| select(.eventName == "GetSecretValue")|[.eventTime, .errorMessage, .userIdentity.arn, .sourceIPAddress, .eventName, .userAgent, .awsRegion]|@csv'

"2020-02-04T10:40:44Z","User: arn:aws:iam::811596193553:user/backup is not authorized to perform: secretsmanager:GetSecretValue on resource: arn:aws:secretsmanager:us-west-2:811596193553:secret:AppEncryptionKey-73a0iL","arn:aws:iam::811596193553:user/backup","250.255.33.75","GetSecretValue","Boto3/1.11.9 Python/3.7.3 Darwin/18.7.0 Botocore/1.14.9","us-west-2"


# Attacker created permament IAM key/user


In [10]:
!gunzip -c *.json.gz | jq -cr '.Records[]| select(.eventName == "CreateAccessKey" and .userIdentity.type == "IAMUser")|[.sourceIPAddress, .userIdentity.arn, .responseElements.accessKey.createDate, .responseElements.accessKey.status, .responseElements.accessKey.accessKeyId, .errorCode, .errorMessage]|@csv'

"204.252.37.154","arn:aws:iam::811596193553:user/Level6",,,,"AccessDenied","User: arn:aws:iam::811596193553:user/Level6 is not authorized to perform: iam:CreateAccessKey on resource: user backup"
"204.252.37.154","arn:aws:iam::811596193553:user/Level6",,,,"AccessDenied","User: arn:aws:iam::811596193553:user/Level6 is not authorized to perform: iam:CreateAccessKey on resource: user backup"
"204.252.37.154","arn:aws:iam::811596193553:user/Level6",,,,"AccessDenied","User: arn:aws:iam::811596193553:user/Level6 is not authorized to perform: iam:CreateAccessKey on resource: user backup"
"251.53.128.202","arn:aws:iam::811596193553:user/backup",,,,"AccessDenied","User: arn:aws:iam::811596193553:user/backup is not authorized to perform: iam:CreateAccessKey on resource: user null"
"224.223.84.255","arn:aws:iam::811596193553:user/piper","Sep 17, 2017 2:59:23 PM","Active","AKIAZY34U15OLS42SFCE",,
"224.223.84.255","arn:aws:iam::811596193553:user/piper","Sep 17, 2017 2:59:54 PM","Active","AKIA4FV6TQ